## YOLOv4 학습하고 Edge TPU complie하기 

### DataSet 생성

In [2]:
## data convert : json to txt
## 데이터 세트를 학습에 적합한 txt로 변환 

import json
from collections import OrderedDict
from tqdm import tqdm

INSTANCES_PATH = "/home/piai/yolov4-tiny/data_anno/_annotations.coco.json"
NAMES_PATH = "/home/piai/yolov4-tiny/data_anno/custom.names"
OUTPUT_FILE_PATH = "/home/piai/yolov4-tiny/data_anno/custom_train.txt"

coco = json.load(open(INSTANCES_PATH))
images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]
replaced_name = {
    "couch": "sofa",
    "airplane": "aeroplane",
    "tv": "tvmonitor",
    "motorcycle": "motorbike",
}

class_to_id = {}
id_to_class = {}
with open(NAMES_PATH, "r") as fd:
    index = 0
    for class_name in fd:
        class_name = class_name.strip()
        if len(class_name) != 0:
            id_to_class[index] = class_name
            class_to_id[class_name] = index
            index += 1

dataset = {}

for annotation in tqdm(annotations, desc="Parsing"):
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]

    # Find image
    file_name = None
    image_height = 0
    image_width = 0
    for image in images:
        if image["id"] == image_id:
            file_name = image["file_name"]
            image_height = image["height"]
            image_width = image["width"]
            break

    if file_name is None:
        continue

    # Find class id
    class_id = None
    for category in categories:
        if category["id"] == category_id:
            category_name = category["name"]
            if category_name in replaced_name:
                category_name = replaced_name[category_name]

            class_id = class_to_id.get(category_name)
            break

    if class_id is None:
        continue

    # Calculate x,y,w,h
    x_center = annotation["bbox"][0] + annotation["bbox"][2] / 2
    x_center /= image_width
    y_center = annotation["bbox"][1] + annotation["bbox"][3] / 2
    y_center /= image_height
    width = annotation["bbox"][2] / image_width
    height = annotation["bbox"][3] / image_height

    if dataset.get(image_id):
        dataset[image_id][1].append(
            [class_id, x_center, y_center, width, height]
        )
    else:
        dataset[image_id] = [
            file_name,
            [[class_id, x_center, y_center, width, height]],
        ]

dataset = OrderedDict(sorted(dataset.items()))

with open(OUTPUT_FILE_PATH, "w") as fd:
    for image_id, bboxes in tqdm(dataset.items(), desc="Saving"):
        data = bboxes[0]
        for bbox in bboxes[1]:
            data += " "
            data += "{:d},".format(bbox[0])
            data += "{:8.6f},".format(bbox[1])
            data += "{:8.6f},".format(bbox[2])
            data += "{:8.6f},".format(bbox[3])
            data += "{:8.6f}".format(bbox[4])

        data += "\n"
        fd.write(data)

Saving: 100%|██████████| 3609/3609 [00:00<00:00, 192248.26it/s]


In [4]:
##training 
## weight 파일 저장하는곳 
## --> https://wiki.loliot.net/docs/lang/python/libraries/yolov4/python-yolov4-about/ 


In [1]:
from yolov4.tf import YOLOv4, YOLODataset, save_as_tflite

Call tf.config.experimental.set_memory_growth(GPU0, True)


### Model Train (GPU 켜고 시작하기)

- 데이터 초기에 생성하고 모델만 하이퍼 파라미터 고쳐가면서 테스트 할때는 여기부터 돌리면 됨

- 커널이 자꾸 죽는다면 뭔가 이상한것 (재부팅하고 학습하더라도 성능이 좋지 않으니 왜 오류 나는지 원인 찾아서 고치고 학습하기)
- 이번 프로젝트에서는 cfg파일과 weight가 안맞아서 생긴 문제였음 

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14585165190811375393,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7601236224
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 848542186437093552
 physical_device_desc: "device: 0, name: GeForce RTX 2080, pci bus id: 0000:17:00.0, compute capability: 7.5",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 7223296672
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 201580105044240433
 physical_device_desc: "device: 1, name: GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.5"]

In [16]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0:1], 'GPU')

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[1], True)

In [ ]:
## Train
## epoch = 240000 / lr = 0.0003 / burn_in = 2000
from tensorflow.keras import callbacks

from yolov4.tf import YOLOv4, YOLODataset, SaveWeightsCallback

yolo = YOLOv4()
yolo.config.parse_names("data/custom.names")
yolo.config.parse_cfg("config/yolov4-tiny-relu.cfg")

yolo.make_model()
yolo.load_weights(
    "yolov4-tiny.conv.29",
    weights_type="yolo",
)
yolo.summary(summary_type="yolo")

for i in range(29):
    yolo.model.get_layer(index=i).trainable = False
    
yolo.summary()

train_dataset = YOLODataset(
    config=yolo.config,
    dataset_list = "data/custom_train_true.txt",
    image_path_prefix="data/train",
    training=True,
)

'''
val_dataset = YOLODataset(
    config=yolo.config,
    dataset_list="/home/piai/yolov4-tiny/data_anno/custom_validation.txt",
    image_path_prefix="/home/piai/yolov4-tiny/data_anno/train",
    training=False,
)
'''

yolo.compile()

_callbacks = [
    SaveWeightsCallback(
        yolo=yolo,
        #dir_path="/home/piai/tensorflow-yolov4-master/trained",
        weights_type="yolo",
        step_per_save=5000,
    ),
]

yolo.fit(
    train_dataset,
    callbacks=_callbacks,
    #validation_data=val_dataset,
    verbose=3,  # 3: print step info
)

In [ ]:
'''
학습결과 
Avg IoU - 현재의 subdivision에서 이미지 평균 IoU
class - 1에 가까울수록 학습 잘되고있는것
No Obj - 0이 아닌 작은 값이어야 함 
.5R - recall/count 
.75R - 0.0000
count -현재 subdivision 이미지들에서 positive sample을 포함한 이미지의 수

'''

### weights 파일 변환하기 

In [23]:
'''
.weights --> .tflite 변환
모델 생성시 사용한 cfg 파일의 conv 옵션과 변환하려는 모델 cfg 파일의 conv 옵션을 맞춰줘야 함 (filter 개수) 

'''

from yolov4.tf import YOLOv4, YOLODataset, save_as_tflite


yolo = YOLOv4()
yolo.config.parse_names("data/custom.names")
yolo.config.parse_cfg("config/yolov4-tiny-relu-tpu.cfg")

yolo.make_model()
yolo.load_weights(
    "0415trained-weights/yolov4-tiny-relu-final.weights", weights_type="yolo"
)

dataset = YOLODataset(
    config=yolo.config,
    dataset_list = "data/custom_train_true.txt",
    image_path_prefix="data/train",
    training=False
)

save_as_tflite(
    model=yolo.model,
    tflite_path="0415_yolov4-tiny-relu-int8.tflite",
    quantization="full_int8",
    dataset=dataset
)

INFO:tensorflow:Assets written to: /tmp/tmpyjw7fhz_/assets


INFO:tensorflow:Assets written to: /tmp/tmpyjw7fhz_/assets


### Edge TPU complier 사용하기 

In [24]:
! edgetpu_compiler  '0415_yolov4-tiny-relu-int8.tflite'

Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 799 ms.

Input model: 0415_yolov4-tiny-relu-int8.tflite
Input size: 5.73MiB
Output model: 0415_yolov4-tiny-relu-int8_edgetpu.tflite
Output size: 5.76MiB
On-chip memory used for caching model parameters: 4.82MiB
On-chip memory remaining for caching model parameters: 1.88MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 51
Operation log: 0415_yolov4-tiny-relu-int8_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 41
Number of operations that will run on CPU: 10
See the operation log file for individual operation details.
